In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [6]:
!pip install google-api-python-client
!pip install google-auth google-auth-oauthlib google-auth-httplib2
!pip install demoji
!pip install pandas
!pip install langdetect


In [ ]:
pip install google-auth google-auth-oauthlib google-auth-httplib2


In [ ]:
pip install langdetect


In [7]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import demoji
from langdetect import detect
import re   # regular expression

In [8]:
import pandas as pd


In [12]:
CLIENT_SECRETS_FILE = "client_secret.json"


In [14]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'


In [15]:
def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [16]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=347867505874-h5isugm7qip3n9tnvtf6s3utvcd3g49l.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=uPUIwAw59KD92zvmwDR9PWxPws4fFz&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g5AHXBJ5yHAXFLB8ASMu8riBHAx8qoitn0O6uvpqkDnzFCsuGVFFFg


Set the search query i.e. video of which comments will be extracted¶


In [112]:
#query= "Shark Tank Việt Nam 4 - Shark Liên bắt gặp Will 365 đi gọi vốn cùng startup Vua Cua"
query="Liên Khúc Một Lần Dang Dở 2020"
#query = " Tầng Lớp Sinh Viên - Tập 22 (Tập Cuối) - Phim Sinh Viên - Tình Cảm | Hi Team - FAPtv"
#https://www.youtube.com/watch?v=hGsEr8KABhU

In [113]:
query_results = service.search().list(part = 'snippet',q = query,
                                      order = 'relevance', 
                                      type = 'video',
                                      relevanceLanguage = 'vi',
                                      safeSearch = 'moderate').execute()

In [ ]:
#query_results['items']


**Extract video details i.e. videoID, channel Name, videoTitle, videoDescription¶
**

In [114]:
video_id = []
channel = []
video_title = []
video_desc = []
for item in query_results['items']:
    video_id.append(item['id']['videoId'])
    channel.append(item['snippet']['channelTitle'])
    video_title.append(item['snippet']['title'])
    video_desc.append(item['snippet']['description'])

We only wants to work with the one video so select the first element in list and save it


In [115]:
video_id = video_id[0]
channel = channel[0]
video_title = video_title[0]
video_desc = video_desc[0]

Extract comments

In [116]:
video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []


comments_temp = []
comment_id_temp = []
reply_count_temp = []
like_count_temp = []

In [119]:
nextPage_token = None

while 1:

  response = service.commentThreads().list(
                    part = 'snippet',
                    videoId = video_id,
                    maxResults = 100, 
                    order = 'relevance', 
                    textFormat = 'plainText',
                    pageToken = nextPage_token).execute()

  nextPage_token = response.get('nextPageToken')
  for item in response['items']:
      comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
      comment_id_temp.append(item['snippet']['topLevelComment']['id'])
      reply_count_temp.append(item['snippet']['totalReplyCount'])
      like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
      comments_pop.extend(comments_temp)
      comment_id_pop.extend(comment_id_temp)
      reply_count_pop.extend(reply_count_temp)
      like_count_pop.extend(like_count_temp)
        
      video_id_pop.extend([video_id]*len(comments_temp))
      channel_pop.extend([channel]*len(comments_temp))
      video_title_pop.extend([video_title]*len(comments_temp))
      video_desc_pop.extend([video_desc]*len(comments_temp))

  if nextPage_token is  None:
    break

In [ ]:
comments_pop

As per now comments have been extracted from youtube video and now we will save the comments into pandas dataframe and then write in csv file

In [121]:
output_dict = {
        
        'Comment': comments_temp,
        'Comment ID': comment_id_temp,
  
        }

output_df = pd.DataFrame(output_dict, columns = output_dict.keys())

In [126]:
output_df.head()


,Comment,Comment ID
0,Nghe 2 ngày liền r😂,UgzZy_jjImTDvWhsqqV4AaABAg
1,Có ai 2k cùng sở thích nghe nhạc theo thế loại...,UgxSl2hjvJ6Cev-DHKt4AaABAg
2,Lâu lâu mở lại nghe vẫn chill vcl 🥰,UgwYKyAzbrQ35Pb_aXF4AaABAg
3,0:00 Một Lần Dang Dở\r\n1:43 Ai khổ Vì Ai\r\n5...,UgzAzF-kjWZKHgf0MQZ4AaABAg
4,0:52,Ugwnm7OASKSZgOdcjM14AaABAg


(**)


In [127]:
output_df.to_csv('coment.csv',encoding='utf-8',index= False)

In [128]:
duplicates = output_df[output_df.duplicated("Comment ID")]
print("Count of duplicat comments in dataframe", duplicates.shape[0])

Count of duplicat comments in dataframe 0


In [ ]:
print("count of unique comments in dataframe ",output_df.shape[0]- duplicates.shape[0])


In [130]:
unique_df = output_df.drop_duplicates(subset=['Comment'])


In [ ]:
unique_df.shape


In [ ]:
unique_df.head(20)


In [132]:
unique_df.to_csv("extraced_comments.csv",encoding='utf-8',index = False)


Data cleaning

In [133]:
comments = pd.read_csv('extraced_comments.csv')


In [ ]:
comments.head()


In [136]:
demoji.download_codes()


... OK (Got response in 0.13 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [137]:
comments['clean_comments'] = comments['Comment'].apply(lambda x: demoji.replace(x,""))


In [138]:
comments.head()

,Comment,Comment ID,clean_comments
0,Nghe 2 ngày liền r😂,UgzZy_jjImTDvWhsqqV4AaABAg,Nghe 2 ngày liền r
1,Có ai 2k cùng sở thích nghe nhạc theo thế loại...,UgxSl2hjvJ6Cev-DHKt4AaABAg,Có ai 2k cùng sở thích nghe nhạc theo thế loại...
2,Lâu lâu mở lại nghe vẫn chill vcl 🥰,UgwYKyAzbrQ35Pb_aXF4AaABAg,Lâu lâu mở lại nghe vẫn chill vcl
3,0:00 Một Lần Dang Dở\r\n1:43 Ai khổ Vì Ai\r\n5...,UgzAzF-kjWZKHgf0MQZ4AaABAg,0:00 Một Lần Dang Dở\r\n1:43 Ai khổ Vì Ai\r\n5...
4,0:52,Ugwnm7OASKSZgOdcjM14AaABAg,0:52


In [139]:
from langdetect import detect


 add new feature language inthe dataframe which signify the language of particular comment


In [140]:
comments['language'] = 0


In [141]:
count = 0
for i in range(0,len(comments)):


  temp = comments['clean_comments'].iloc[i]
  count += 1
  try:
    comments['language'].iloc[i] = detect(temp)
  except:
    comments['language'].iloc[i] = "error"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [142]:
comments[comments['language']=='vi']['language'].value_counts()


vi    707
Name: language, dtype: int64

In [143]:
vi_comm = comments[comments['language'] == 'vi']


In [144]:
vi_comm.to_csv("vi2_comments.csv",index = False)


In [145]:
vi_comments = pd.read_csv('vi2_comments.csv',encoding='utf-8',error_bad_lines=False)


In [146]:
vi_comments.head()


,Comment,Comment ID,clean_comments,language
0,Nghe 2 ngày liền r😂,UgzZy_jjImTDvWhsqqV4AaABAg,Nghe 2 ngày liền r,vi
1,Có ai 2k cùng sở thích nghe nhạc theo thế loại...,UgxSl2hjvJ6Cev-DHKt4AaABAg,Có ai 2k cùng sở thích nghe nhạc theo thế loại...,vi
2,Lâu lâu mở lại nghe vẫn chill vcl 🥰,UgwYKyAzbrQ35Pb_aXF4AaABAg,Lâu lâu mở lại nghe vẫn chill vcl,vi
3,0:00 Một Lần Dang Dở\r\n1:43 Ai khổ Vì Ai\r\n5...,UgzAzF-kjWZKHgf0MQZ4AaABAg,0:00 Một Lần Dang Dở\r\n1:43 Ai khổ Vì Ai\r\n5...,vi
4,Ghi nhớ ngày nay 22/6 nào vô xem lại 😭😭😭,UgzDYfPR7wXvZQ2ft5h4AaABAg,Ghi nhớ ngày nay 22/6 nào vô xem lại,vi


In [147]:

vi_comments.shape


(707, 4)

In [148]:
regex= r"\b\S*[AĂÂÁẮẤÀẰẦẢẲẨÃẴẪẠẶẬĐEÊÉẾÈỀẺỂẼỄẸỆIÍÌỈĨỊOÔƠÓỐỚÒỒỜỎỔỞÕỖỠỌỘỢUƯÚỨÙỪỦỬŨỮỤỰYÝỲỶỸỴAĂÂÁẮẤÀẰẦẢẲẨÃẴẪẠẶẬĐEÊÉẾÈỀẺỂẼỄẸỆIÍÌỈĨỊOÔƠÓỐỚÒỒỜỎỔỞÕỖỠỌỘỢUƯÚỨÙỪỦỬŨỮỤỰYÝỲỶỸỴAĂÂÁẮẤÀẰẦẢẲẨÃẴẪẠẶẬĐEÊÉẾÈỀẺỂẼỄẸỆIÍÌỈĨỊOÔƠÓỐỚÒỒỜỎỔỞÕỖỠỌỘỢUƯÚỨÙỪỦỬŨỮỤỰYÝỲỶỸỴAĂÂÁẮẤÀẰẦẢẲẨÃẴẪẠẶẬĐEÊÉẾÈỀẺỂẼỄẸỆIÍÌỈĨỊOÔƠÓỐỚÒỒỜỎỔỞÕỖỠỌỘỢUƯÚỨÙỪỦỬŨỮỤỰYÝỲỶỸỴAĂÂÁẮẤÀẰẦẢẲẨÃẴẪẠẶẬĐEÊÉẾÈỀẺỂẼỄẸỆIÍÌỈĨỊOÔƠÓỐỚÒỒỜỎỔỞÕỖỠỌỘỢUƯÚỨÙỪỦỬŨỮỤỰYÝỲỶỸỴAĂÂÁẮẤÀẰẦẢẲẨÃẴẪẠẶẬĐEÊÉẾÈỀẺỂẼỄẸỆIÍÌỈĨỊOÔƠÓỐỚÒỒỜỎỔỞÕỖỠỌỘỢUƯÚỨÙỪỦỬŨỮỤỰYÝỲỶỸỴA-Z]+\S*\b"


In [149]:
copy = vi_comments.copy()


In [150]:
copy['reg'] = copy['clean_comments'].apply(lambda x:re.findall(regex,x))
copy['regular_comments'] = copy['clean_comments'].apply(lambda x:re.sub(regex,"  ",x))

In [151]:
copy.head(20)

,Comment,Comment ID,clean_comments,language,reg,regular_comments
0,Nghe 2 ngày liền r😂,UgzZy_jjImTDvWhsqqV4AaABAg,Nghe 2 ngày liền r,vi,[Nghe],2 ngày liền r
1,Có ai 2k cùng sở thích nghe nhạc theo thế loại...,UgxSl2hjvJ6Cev-DHKt4AaABAg,Có ai 2k cùng sở thích nghe nhạc theo thế loại...,vi,[Có],ai 2k cùng sở thích nghe nhạc theo thế loại...
2,Lâu lâu mở lại nghe vẫn chill vcl 🥰,UgwYKyAzbrQ35Pb_aXF4AaABAg,Lâu lâu mở lại nghe vẫn chill vcl,vi,[Lâu],lâu mở lại nghe vẫn chill vcl
3,0:00 Một Lần Dang Dở\r\n1:43 Ai khổ Vì Ai\r\n5...,UgzAzF-kjWZKHgf0MQZ4AaABAg,0:00 Một Lần Dang Dở\r\n1:43 Ai khổ Vì Ai\r\n5...,vi,"[Một, Lần, Dang, Dở, Ai, Vì, Ai, Gặp, Nhau, Là...",0:00 \r\n1:43 khổ \r\n5:30 ...
4,Ghi nhớ ngày nay 22/6 nào vô xem lại 😭😭😭,UgzDYfPR7wXvZQ2ft5h4AaABAg,Ghi nhớ ngày nay 22/6 nào vô xem lại,vi,[Ghi],nhớ ngày nay 22/6 nào vô xem lại
5,06:35 Muốn có một mối quan hệ mập mờ … 🔼,UgyApZ-VENA9ZOT3_ph4AaABAg,06:35 Muốn có một mối quan hệ mập mờ …,vi,[Muốn],06:35 có một mối quan hệ mập mờ …
6,đang đợi nhanh đến tết để mở mấy bài này :)),UgywH93Ns3RKAzNAW9V4AaABAg,đang đợi nhanh đến tết để mở mấy bài này :)),vi,[],đang đợi nhanh đến tết để mở mấy bài này :))
7,04:16 Còn anh thì chỉ của mình em thôi. 🐧🕊,UgxZlQMybuvRRetsnuZ4AaABAg,04:16 Còn anh thì chỉ của mình em thôi.,vi,[Còn],04:16 anh thì chỉ của mình em thôi.
8,Mình thích thể loại này. Nhạc trữ tình nhưng p...,UgxDaexXFTD0aQQ1MKl4AaABAg,Mình thích thể loại này. Nhạc trữ tình nhưng p...,vi,"[Mình, Nhạc]",thích thể loại này. trữ tình nhưng phối ...
9,"mấy bài này có độ bánh cuốn ghê cơ ý ^^, thỉnh...",UgxONwbz0FRZ1Q3pm8N4AaABAg,"mấy bài này có độ bánh cuốn ghê cơ ý ^^, thỉnh...",vi,[],"mấy bài này có độ bánh cuốn ghê cơ ý ^^, thỉnh..."


In [152]:
dataset = copy[['Comment ID','regular_comments']].copy()


In [153]:
dataset.head()


,Comment ID,regular_comments
0,UgzZy_jjImTDvWhsqqV4AaABAg,2 ngày liền r
1,UgxSl2hjvJ6Cev-DHKt4AaABAg,ai 2k cùng sở thích nghe nhạc theo thế loại...
2,UgwYKyAzbrQ35Pb_aXF4AaABAg,lâu mở lại nghe vẫn chill vcl
3,UgzAzF-kjWZKHgf0MQZ4AaABAg,0:00 \r\n1:43 khổ \r\n5:30 ...
4,UgzDYfPR7wXvZQ2ft5h4AaABAg,nhớ ngày nay 22/6 nào vô xem lại


In [154]:
dataset = dataset.rename(columns = {"regular_comments":"comments"})


In [155]:
dataset.head(20)


,Comment ID,comments
0,UgzZy_jjImTDvWhsqqV4AaABAg,2 ngày liền r
1,UgxSl2hjvJ6Cev-DHKt4AaABAg,ai 2k cùng sở thích nghe nhạc theo thế loại...
2,UgwYKyAzbrQ35Pb_aXF4AaABAg,lâu mở lại nghe vẫn chill vcl
3,UgzAzF-kjWZKHgf0MQZ4AaABAg,0:00 \r\n1:43 khổ \r\n5:30 ...
4,UgzDYfPR7wXvZQ2ft5h4AaABAg,nhớ ngày nay 22/6 nào vô xem lại
5,UgyApZ-VENA9ZOT3_ph4AaABAg,06:35 có một mối quan hệ mập mờ …
6,UgywH93Ns3RKAzNAW9V4AaABAg,đang đợi nhanh đến tết để mở mấy bài này :))
7,UgxZlQMybuvRRetsnuZ4AaABAg,04:16 anh thì chỉ của mình em thôi.
8,UgxDaexXFTD0aQQ1MKl4AaABAg,thích thể loại này. trữ tình nhưng phối ...
9,UgxONwbz0FRZ1Q3pm8N4AaABAg,"mấy bài này có độ bánh cuốn ghê cơ ý ^^, thỉnh..."


In [156]:
dataset.to_csv("Dataset2.csv",index = False)
